# Tutorial 9: Neural Networks

<img src="../../docs/imgs/lightautoml_logo_color.png" alt="LightAutoML logo" style="width:100%;"/>

Official LightAutoML github repository is [here](https://github.com/AILab-MLTools/LightAutoML)


In this tutorial you will learn how to:
* train neural networks (nn) with LightAutoML on tabualr data
* customize model architecture and pipelines

## 0. Prerequisites

### 0.0 install LightAutoML

In [ ]:
# !pip install -U lightautoml[all]

### 0.1 Import libraries

Here we will import the libraries we use in this kernel:
- Standard python libraries for timing, working with OS etc.
- Essential python DS libraries like numpy, pandas, scikit-learn and torch (the last we will use in the next cell)
- LightAutoML modules: presets for AutoML, task and report generation module

In [2]:
# Standard python libraries
import os

# Essential DS libraries
import optuna
import requests
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import torch
from copy import deepcopy as copy
import torch.nn as nn
from collections import OrderedDict

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

/home/dvladimirvasilyev/anaconda3/envs/myenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


/home/dvladimirvasilyev/LightAutoML/lightautoml/ml_algo/dl_model.py:41: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/home/dvladimirvasilyev/LightAutoML/lightautoml/text/nn_model.py:22: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/home/dvladimirvasilyev/LightAutoML/lightautoml/text/dl_transformers.py:25: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")


### 0.2 Constants

Here we setup the constants to use in the kernel:
- `N_THREADS` - number of vCPUs for LightAutoML model creation
- `N_FOLDS` - number of folds in LightAutoML inner CV
- `RANDOM_STATE` - random seed for better reproducibility
- `TEST_SIZE` - houldout data part size 
- `TIMEOUT` - limit in seconds for model to train
- `TARGET_NAME` - target column name in dataset

In [3]:
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2
TIMEOUT = 300
TARGET_NAME = 'TARGET'

np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

### 0.3 Data loading

In [4]:
DATASET_DIR = '../data/'
DATASET_NAME = 'sampled_app_train.csv'
DATASET_FULLNAME = os.path.join(DATASET_DIR, DATASET_NAME)
DATASET_URL = 'https://raw.githubusercontent.com/AILab-MLTools/LightAutoML/master/examples/data/sampled_app_train.csv'

if not os.path.exists(DATASET_FULLNAME):
    os.makedirs(DATASET_DIR, exist_ok=True)

    dataset = requests.get(DATASET_URL).text
    with open(DATASET_FULLNAME, 'w') as output:
        output.write(dataset)

data = pd.read_csv(DATASET_FULLNAME)
data.head()

tr_data, te_data = train_test_split(
    data, 
    test_size=TEST_SIZE, 
    stratify=data[TARGET_NAME], 
    random_state=RANDOM_STATE
)

## 1. Available built-in models

To use different model pass it to the list in `"use_algo"`. We support custom models inherited from `torch.nn.Module` class. For every model their parameters is listed below.

### 1.1 MLP (`"mlp"`)
- `hidden_size` - define hidden layer dimensions

### 1.2 Dense Light (`"denselight"`)
<img src="../../docs/imgs/denselight.png" style="width:25%;"/>

- `hidden_size` - define hidden layer dimensions

### 1.3 Dense (`"dense"`)
<img src="../../docs/imgs/densenet.png" style="width:60%;"/>

- `block_config` - set number of blocks and layers within each block
- `compression` - portion of neuron to drop after `DenseBlock`
- `growth_size` - output dim of every `DenseLayer`
- `bn_factor` - size of intermediate fc is increased times this factor in layer

### 1.4 Resnet (`"resnet"`)
<img src="../../docs/imgs/resnet.png" style="width:50%;"/>

- `hid_factor` - size of intermediate fc is increased times this factor in layer

### 1.5 SNN (`"snn"`)
- `hidden_size` - define hidden layer dimensions

### 1.5 NODE (`"node"`)
<img src="../../docs/imgs/node.png" style="width:80%;"/>

### 1.5 AutoInt (`"autoint"`)
<img src="../../docs/imgs/autoint.png" style="width:80%;"/>

### 1.5 FTTransformer (`"fttransformer"`)
<img src="../../docs/imgs/fttransformer.png" style="width:80%;"/>

- `pooling` - Pooling used for the last step.
- `n_out` - Output dimension, 1 for binary prediction.
- `embedding_size` - Embeddings size.
- `depth` - Number of Attention Blocks inside Transformer.
- `heads` - Number of heads in Attention.
- `attn_dropout` - Post-Attention dropout.
- `ff_dropout` - Feed-Forward Dropout.
- `dim_head` - Attention head dimension
- `return_attn` - Return attention scores or not.
- `num_enc_layers` - Number of Transformer layers.
- `device` - Device to compute on.


## 2. Example of usage
### 2.1 Task definition

In [5]:
task = Task('binary')
roles = {
    'target': TARGET_NAME,
    'drop': ['SK_ID_CURR']
}

### 2.2 LightAutoML model creation - TabularAutoML preset with neural network

In next the cell we are going to create LightAutoML model with `TabularAutoML` class.

in just several lines. Let's discuss the params we can setup:
- `task` - the type of the ML task (the only **must have** parameter)
- `timeout` - time limit in seconds for model to train
- `cpu_limit` - vCPU count for model to use
- `nn_params` - network and training params, for example, `"hidden_size"`, `"batch_size"`, `"lr"`, etc.
- `nn_pipeline_params` - data preprocessing params, which affect how data is fed to the model: use embeddings or target encoding for categorical columns, standard scalar or quantile transformer for numerical columns
- `reader_params` - parameter change for Reader object inside preset, which works on the first step of data preparation: automatic feature typization, preliminary almost-constant features, correct CV setup etc.

In [5]:
automl = TabularAutoML(
    task = task, 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    general_params = {"use_algos": [["mlp"]]}, # ['nn', 'mlp', 'dense', 'denselight', 'resnet', 'snn', 'node', 'autoint', 'fttransformer'] or custom torch model
    nn_params = {"n_epochs": 10, "bs": 512, "num_workers": 0, "path_to_save": None, "freeze_defaults": True},
    nn_pipeline_params = {"use_qnt": True, "use_te": False},
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
)

### 2.3 AutoML training

To run autoML training use fit_predict method:

- `train_data` - Dataset to train.
- `roles` - Roles dict.
- `verbose` - Controls the verbosity: the higher, the more messages.
        <1  : messages are not displayed;
        >=1 : the computation process for layers is displayed;
        >=2 : the information about folds processing is also displayed;
        >=3 : the hyperparameters optimization process is also displayed;
        >=4 : the training process for every algorithm is displayed;

Note: out-of-fold prediction is calculated during training and returned from the fit_predict method

In [6]:
%%time 
oof_pred = automl.fit_predict(tr_data, roles = roles, verbose = 1)

[11:35:05] Stdout logging level is INFO.
[11:35:05] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[11:35:05] Task: binary

[11:35:05] Start automl preset with listed constraints:
[11:35:05] - time: 300.00 seconds
[11:35:05] - CPU: 4 cores
[11:35:05] - memory: 16 GB

[11:35:05] Train data shape: (8000, 122)

[11:35:07] Layer 1 train process start. Time left 297.35 secs
[11:35:08] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_mlp_0 ...
[11:35:20] Fitting Lvl_0_Pipe_0_Mod_0_TorchNN_mlp_0 finished. score = 0.6951557493612979
[11:35:20] Lvl_0_Pipe_0_Mod_0_TorchNN_mlp_0 fitting and predicting completed
[11:35:20] Time left 284.31 secs

[11:35:20] Layer 1 training completed.

[11:35:20] Automl preset training completed in 15.70 seconds

[11:35:20] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (5 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN_mlp_0) 

CPU times: user 15.8 s, sys: 1.58 s, total: 17.4 s
Wall time: 15.7 s


### 2.4 Prediction on holdout and model evaluation

In [7]:
%%time

te_pred = automl.predict(te_data)
print(f'Prediction for te_data:\n{te_pred}\nShape = {te_pred.shape}')

Prediction for te_data:
array([[0.08216639],
       [0.08314921],
       [0.07000729],
       ...,
       [0.07061756],
       [0.09196799],
       [0.16275021]], dtype=float32)
Shape = (2000, 1)
CPU times: user 1.07 s, sys: 30.4 ms, total: 1.1 s
Wall time: 1 s


In [8]:
print(f'OOF score: {roc_auc_score(tr_data[TARGET_NAME].values, oof_pred.data[:, 0])}')
print(f'HOLDOUT score: {roc_auc_score(te_data[TARGET_NAME].values, te_pred.data[:, 0])}')

OOF score: 0.6951557493612979
HOLDOUT score: 0.7132812500000001


You can obtain the description of the resulting pipeline:

In [9]:
print(automl.create_model_str_desc())

Final prediction for new objects (level 0) = 
	 1.00000 * (5 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN_mlp_0) 


## 3. Main training loop and pipeline params

### 3.1 Training loop params

<img src="../../docs/imgs/swa.png" style="width:70%;"/>

- `bs` - batch_size
- `snap_params` - early stopping and checkpoint averaging params, stochastic weight averaging (swa)
- `opt` - lr optimizer
- `opt_params` - optimizer params
- `clip_grad` - use grad clipping for regularization
- `clip_grad_params`
- `emb_dropout` - embedding dropout for categorical columns

This set of params should be passed in `nn_params` as well.

### 3.2 Pipeline params

Transformation for numerical columns

- `use_qnt` - uses quantile transformation for numerical columns
- `output_distribution` - type of distribuiton of feature after qnt transformer
- `n_quantiles` - number of quantiles used to build feature distribution
- `qnt_factor` - decreses `n_quantiles` depending on train data shape

Transformation for categorical columns

- `use_te` - uses target encoding
- `top_intersections` - number of intersections of cat columns to use

Full list of default parametres you can find here:
- [nn_params](../../lightautoml/automl/presets/tabular_config.yml)
- [nn_pipeline_params](../../lightautoml/automl/presets/tabular_config.yml)

## 4. More use cases

Let's remember default Lama params to be more compact.

In [5]:
default_lama_params = {
    "task": task, 
    "timeout": TIMEOUT,
    "cpu_limit": N_THREADS,
    "reader_params": {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
}

default_nn_params = {
    "bs": 512, "num_workers": 0, "path_to_save": None, "n_epochs": 10, "freeze_defaults": True
}

### 4.1 Custom model

Consider simple neural network that we want to train. 

In [11]:
class SimpleNet(nn.Module):
    def __init__(
        self,
        n_in,
        n_out,
        hidden_size,
        drop_rate,
        **kwargs, # kwargs is must-have to hold unnecessary parameters
    ):
        super(SimpleNet, self).__init__()
        self.features = nn.Sequential(OrderedDict([]))

        self.features.add_module("norm", nn.BatchNorm1d(n_in))
        self.features.add_module("dense1", nn.Linear(n_in, hidden_size))
        self.features.add_module("act", nn.SiLU())
        self.features.add_module("dropout", nn.Dropout(p=drop_rate))
        self.features.add_module("dense2", nn.Linear(hidden_size, n_out))

    def forward(self, x):
        """
        Args:
            x: data after feature pipeline transformation
            (by default concatenation of columns)
        """
        for layer in self.features:
            x = layer(x)
        return x

In [12]:
automl = TabularAutoML(
    **default_lama_params,
    general_params={"use_algos": [[SimpleNet]]},
    nn_params={
        **default_nn_params,
        "hidden_size": 256,
        "drop_rate": 0.1
    },
)
automl.fit_predict(tr_data, roles=roles, verbose=1)


[11:39:19] Stdout logging level is INFO.
[11:39:19] Task: binary

[11:39:19] Start automl preset with listed constraints:
[11:39:19] - time: 300.00 seconds
[11:39:19] - CPU: 4 cores
[11:39:19] - memory: 16 GB

[11:39:19] Train data shape: (8000, 122)

[11:39:20] Layer 1 train process start. Time left 299.14 secs
[11:39:20] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_0 ...
[11:39:29] Fitting Lvl_0_Pipe_0_Mod_0_TorchNN_0 finished. score = 0.7060418025974987
[11:39:29] Lvl_0_Pipe_0_Mod_0_TorchNN_0 fitting and predicting completed
[11:39:29] Time left 290.88 secs

[11:39:29] Layer 1 training completed.

[11:39:29] Automl preset training completed in 9.12 seconds

[11:39:29] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (5 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN_0) 



array([[0.02449569],
       [0.03754642],
       [0.04070117],
       ...,
       [0.06268083],
       [0.19106267],
       [0.13282676]], dtype=float32)

#### 4.1.1 Define the pipeline by yourself

In [13]:
from typing import Sequence
from typing import Dict
from typing import Optional
from typing import Any
from typing import Callable
from typing import Union


class CatEmbedder(nn.Module):
    """Category data model.

    Args:
        cat_dims: Sequence with number of unique categories
            for category features
    """

    def __init__(
        self,
        cat_dims: Sequence[int],
        **kwargs
    ):
        super(CatEmbedder, self).__init__()
        emb_dims = [
            (int(x), 5)
            for x in cat_dims
        ]
        self.no_of_embs = sum([y for x, y in emb_dims])
        self.emb_layers = nn.ModuleList([nn.Embedding(x, y) for x, y in emb_dims])
    
    def get_out_shape(self) -> int:
        """Output shape.

        Returns:
            Int with module output shape.

        """
        return self.no_of_embs

    def forward(self, inp: Dict[str, torch.Tensor]) -> torch.Tensor:
        """Concat all categorical embeddings
        """
        output = torch.cat(
            [
                emb_layer(inp["cat"][:, i])
                for i, emb_layer in enumerate(self.emb_layers)
            ],
            dim=1,
        )
        return output


class ContEmbedder(nn.Module):
    """Numeric data model.

    Class for working with numeric data.

    Args:
        num_dims: Sequence with number of numeric features.
        input_bn: Use 1d batch norm for input data.

    """

    def __init__(self, num_dims: int,  **kwargs):
        super(ContEmbedder, self).__init__()
        self.n_out = num_dims
    
    def get_out_shape(self) -> int:
        """Output shape.

        Returns:
            int with module output shape.

        """
        return self.n_out
        
    def forward(self, inp: Dict[str, torch.Tensor]) -> torch.Tensor:
        """Forward-pass."""
        return (inp["cont"] - inp["cont"].mean(axis=0)) / (inp["cont"].std(axis=0) + 1e-6)

In [14]:
from lightautoml.text.nn_model import TorchUniversalModel

class SimpleNet_plus(TorchUniversalModel):
    """Mixed data model.

    Class for preparing input for DL model with mixed data.

    Args:
            n_out: Number of output dimensions.
            cont_params: Dict with numeric model params.
            cat_params: Dict with category model para
            **kwargs: Loss, task and other parameters.

        """

    def __init__(
            self,
            n_out: int = 1,
            cont_params: Optional[Dict] = None,
            cat_params: Optional[Dict] = None,
            **kwargs,
    ):
        # init parent class (need some helper functions to be used)
        super(SimpleNet_plus, self).__init__(**{
                **kwargs,
                "cont_params": cont_params,
                "cat_params": cat_params,
                "torch_model": None, # dont need any model inside parent class
        })
        
        n_in = 0
        
        # add cont columns processing
        self.cont_embedder = ContEmbedder(**cont_params)
        n_in += self.cont_embedder.get_out_shape()
        
        # add cat columns processing
        self.cat_embedder = CatEmbedder(**cat_params)
        n_in += self.cat_embedder.get_out_shape()
        
        self.torch_model = SimpleNet(
                **{
                    **kwargs,
                    **{"n_in": n_in, "n_out": n_out},
                }
        )
    
    def get_logits(self, inp: Dict[str, torch.Tensor]) -> torch.Tensor:
        outputs = []
        outputs.append(self.cont_embedder(inp))
        outputs.append(self.cat_embedder(inp))
        
        if len(outputs) > 1:
            output = torch.cat(outputs, dim=1)
        else:
            output = outputs[0]
        
        logits = self.torch_model(output)
        return logits

In [15]:
automl = TabularAutoML(
    **default_lama_params,
    general_params={"use_algos": [[SimpleNet_plus]]},
    nn_params={
        **default_nn_params,
        "hidden_size": 256,
        "drop_rate": 0.1,
        "model_with_emb": True,
    },
    debug=True
)
automl.fit_predict(tr_data, roles = roles, verbose = 1)

[11:39:33] Stdout logging level is INFO.
[11:39:33] Task: binary

[11:39:33] Start automl preset with listed constraints:
[11:39:33] - time: 300.00 seconds
[11:39:33] - CPU: 4 cores
[11:39:33] - memory: 16 GB

[11:39:33] Train data shape: (8000, 122)

[11:39:34] Layer 1 train process start. Time left 299.14 secs
[11:39:34] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_0 ...
[11:39:42] Fitting Lvl_0_Pipe_0_Mod_0_TorchNN_0 finished. score = 0.680797945608108
[11:39:42] Lvl_0_Pipe_0_Mod_0_TorchNN_0 fitting and predicting completed
[11:39:42] Time left 290.91 secs

[11:39:42] Layer 1 training completed.

[11:39:42] Automl preset training completed in 9.10 seconds

[11:39:42] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (5 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN_0) 



array([[0.06662331],
       [0.05009553],
       [0.05109952],
       ...,
       [0.07657926],
       [0.19059831],
       [0.04237348]], dtype=float32)

### 4.2 Tuning network

One can try optimize metric with the help of Optuna. Among validation stratagies there are:
- `fit_on_holdout = True` - holdout
- `fit_on_holdout = False` - cross-validation.

#### 4.2.1 Built-in models

Use `"_tuned"` in model name to tune it.

In [17]:
automl = TabularAutoML(
    **default_lama_params,
    general_params={"use_algos": [["denselight_tuned"]]},
    nn_params={
        **default_nn_params,
        "n_epochs": 3,
        "tuning_params": {
            "max_tuning_iter": 5,
            "max_tuning_time": 100,
            "fit_on_holdout": True
        }
    },
)
automl.fit_predict(tr_data, roles = roles, verbose = 3)

[11:41:13] Stdout logging level is INFO3.
[11:41:13] Task: binary

[11:41:13] Start automl preset with listed constraints:
[11:41:13] - time: 300.00 seconds
[11:41:13] - CPU: 4 cores
[11:41:13] - memory: 16 GB

[11:41:13] Train data shape: (8000, 122)

[11:41:14] Feats was rejected during automatic roles guess: []
[11:41:14] Layer 1 train process start. Time left 299.15 secs
[11:41:14] Start hyperparameters optimization for Lvl_0_Pipe_0_Mod_0_Tuned_TorchNN_denselight_tuned_0 ... Time budget is 100.00 secs
[11:41:15] Epoch: 0, train loss: 1.0535998344421387, val loss: 0.32914862036705017, val metric: 0.6417082284266402
[11:41:16] Epoch: 1, train loss: 0.2719154357910156, val loss: 0.29687464237213135, val metric: 0.7061383111225151
[11:41:16] Epoch: 2, train loss: 0.2606324255466461, val loss: 0.26732537150382996, val metric: 0.7064643905255223
[11:41:16] Early stopping: val loss: 0.27287718653678894, val metric: 0.7066167390990586
[11:41:16] Trial 1 with hyperparameters {'bs': 128, 'we

array([[0.00909923],
       [0.06779448],
       [0.05014049],
       ...,
       [0.04888163],
       [0.18241519],
       [0.07331596]], dtype=float32)

#### 4.2.2 Custom model

There is a spesial flag `tuned` to mark that you need optimize parameters for the model.

In [18]:
automl = TabularAutoML(
    **default_lama_params,
    general_params={"use_algos": [[SimpleNet]]},
    nn_params={
        **default_nn_params,
        "hidden_size": 256,
        "drop_rate": 0.1,
        
        "tuned": True,
        "tuning_params": {
            "max_tuning_iter": 5,
            "max_tuning_time": 100,
            "fit_on_holdout": True
        }
    },
)
automl.fit_predict(tr_data, roles = roles, verbose = 2)

[11:41:56] Stdout logging level is INFO2.
[11:41:56] Task: binary

[11:41:56] Start automl preset with listed constraints:
[11:41:56] - time: 300.00 seconds
[11:41:56] - CPU: 4 cores
[11:41:56] - memory: 16 GB

[11:41:56] Train data shape: (8000, 122)

[11:41:57] Layer 1 train process start. Time left 299.16 secs
[11:41:57] Start hyperparameters optimization for Lvl_0_Pipe_0_Mod_0_Tuned_TorchNN_0 ... Time budget is 100.00 secs
[11:42:17] Hyperparameters optimization for Lvl_0_Pipe_0_Mod_0_Tuned_TorchNN_0 completed
[11:42:17] The set of hyperparameters {'num_workers': 0, 'pin_memory': False, 'max_length': 256, 'is_snap': False, 'input_bn': False, 'max_emb_size': 256, 'bert_name': None, 'pooling': 'cls', 'device': ['0', '1'], 'use_cont': True, 'use_cat': True, 'use_text': False, 'lang': 'en', 'deterministic': True, 'multigpu': False, 'random_state': 42, 'model': <class '__main__.SimpleNet'>, 'model_with_emb': False, 'path_to_save': None, 'verbose_inside': None, 'verbose': 1, 'n_epochs': 

array([[0.01359685],
       [0.02897443],
       [0.01692689],
       ...,
       [0.04529661],
       [0.17770922],
       [0.17924136]], dtype=float32)

Sometimes we need to tune parameters that we define by ourself. To this purpose we have `optimization_search_space` which describes necessary parameter grid. See example below.  
Here is the grid:  
- `bs` in `[64, 128, 256, 512, 1024]`
- `hidden_size` in `[64, 128, 256, 512, 1024]`
- `drop_rate` in `[0.0, 0.3]`


In [19]:
def my_opt_space(trial: optuna.trial.Trial, estimated_n_trials, suggested_params):
    ''' 
        This function needs for parameter tuning
    '''
    # optionally
    trial_values = copy(suggested_params)

    trial_values["bs"] = trial.suggest_categorical(
        "bs", [2 ** i for i in range(6, 11)]
    )
    trial_values["hidden_size"] = trial.suggest_categorical(
        "hidden_size", [2 ** i for i in range(6, 11)]
    )
    trial_values["drop_rate"] = trial.suggest_float(
        "drop_rate", 0.0, 0.3
    )
    return trial_values

In [20]:
automl = TabularAutoML(
    **default_lama_params,
    general_params={"use_algos": [[SimpleNet]]},
    nn_params={
        **default_nn_params,
        "n_epochs": 3,
        "tuned": True,
        "tuning_params": {
            "max_tuning_iter": 5,
            "max_tuning_time": 3600,
            "fit_on_holdout": True
        },
        "optimization_search_space": my_opt_space,
    },
)
automl.fit_predict(tr_data, roles = roles, verbose = 3)

[11:42:39] Stdout logging level is INFO3.
[11:42:39] Task: binary

[11:42:39] Start automl preset with listed constraints:
[11:42:39] - time: 300.00 seconds
[11:42:39] - CPU: 4 cores
[11:42:39] - memory: 16 GB

[11:42:39] Train data shape: (8000, 122)

[11:42:39] Feats was rejected during automatic roles guess: []
[11:42:40] Layer 1 train process start. Time left 299.16 secs
[11:42:40] Start hyperparameters optimization for Lvl_0_Pipe_0_Mod_0_Tuned_TorchNN_0 ... Time budget is 156.93 secs
[11:42:40] Epoch: 0, train loss: 0.2768203020095825, val loss: 0.27835753560066223, val metric: 0.6358815636843766
[11:42:41] Epoch: 1, train loss: 0.269419401884079, val loss: 0.270462304353714, val metric: 0.6819656707880963
[11:42:41] Epoch: 2, train loss: 0.2603665292263031, val loss: 0.26111170649528503, val metric: 0.7290280161008387
[11:42:41] Early stopping: val loss: 0.2704150676727295, val metric: 0.6917587440062863
[11:42:41] Trial 1 with hyperparameters {'bs': 128, 'hidden_size': 256, 'dro

array([[0.03668038],
       [0.03266167],
       [0.0428489 ],
       ...,
       [0.05952494],
       [0.19782786],
       [0.10605511]], dtype=float32)

##### 4.2.3 One more example
##### Tuning NODE params

In [6]:
TIMEOUT = 3000

In [15]:
default_lama_params = {
    "task": task, 
    "timeout": TIMEOUT,
    "cpu_limit": N_THREADS,
    "reader_params": {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
}

default_nn_params = {
    "bs": 512, "num_workers": 0, "path_to_save": None, "n_epochs": 10, "freeze_defaults": True
}

In [16]:
def my_opt_space_NODE(trial: optuna.trial.Trial, estimated_n_trials, suggested_params):
    ''' 
        This function needs for parameter tuning
    '''
    # optionally
    trial_values = copy(suggested_params)

    trial_values["layer_dim"] = trial.suggest_categorical(
        "layer_dim", [2 ** i for i in range(8, 10)]
    )
    trial_values["use_original_head"] = trial.suggest_categorical(
        "use_original_head", [True, False]
    )
    trial_values["num_layers"] = trial.suggest_int(
        "num_layers", 1, 3
    )
    trial_values["drop_rate"] = trial.suggest_float(
        "drop_rate", 0.0, 0.3
    )
    trial_values["tree_dim"] = trial.suggest_int(
        "tree_dim", 1, 3
    )
    return trial_values

In [17]:
automl = TabularAutoML(
    task = task, 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    general_params = {"use_algos": [["node_tuned"]]}, # ['nn', 'mlp', 'dense', 'denselight', 'resnet', 'snn'] or custom torch model
    nn_params = {"n_epochs": 10, "bs": 512, "num_workers": 0, "path_to_save": None, "freeze_defaults": True, "optimization_search_space": my_opt_space_NODE,},
    nn_pipeline_params = {"use_qnt": True, "use_te": False},
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
)

In [18]:
oof_pred = automl.fit_predict(tr_data, roles = roles, verbose = 2)

[11:58:03] Stdout logging level is INFO2.
[11:58:03] Task: binary

[11:58:03] Start automl preset with listed constraints:
[11:58:03] - time: 3000.00 seconds
[11:58:03] - CPU: 4 cores
[11:58:03] - memory: 16 GB

[11:58:03] Train data shape: (8000, 122)

[11:58:03] Layer 1 train process start. Time left 2999.19 secs
[11:58:04] Start hyperparameters optimization for Lvl_0_Pipe_0_Mod_0_Tuned_TorchNN_node_tuned_0 ... Time budget is 1574.27 secs
[12:01:57] Hyperparameters optimization for Lvl_0_Pipe_0_Mod_0_Tuned_TorchNN_node_tuned_0 completed
[12:01:57] The set of hyperparameters {'num_workers': 0, 'pin_memory': False, 'max_length': 256, 'is_snap': False, 'input_bn': False, 'max_emb_size': 256, 'bert_name': None, 'pooling': 'cls', 'device': ['0'], 'use_cont': True, 'use_cat': True, 'use_text': False, 'lang': 'en', 'deterministic': True, 'multigpu': False, 'random_state': 42, 'model': 'node', 'model_with_emb': False, 'path_to_save': None, 'verbose_inside': None, 'verbose': 1, 'n_epochs': 10

### 4.3 Several models

If you have several neural networks you can either define one set parameters for all or use unique for each one of them as below.  
**Note:** numeration starts with 0. Each id (string of number) corresponds to the serial number in *the list of used neural networks*.

In [21]:
automl = TabularAutoML(
    **default_lama_params,
    general_params = {"use_algos": [["lgb", "mlp", "dense"]]},
    nn_params = {"0": {**default_nn_params, "n_epochs": 2},
                 "1": {**default_nn_params, "n_epochs": 5}},
)
automl.fit_predict(tr_data, roles = roles, verbose = 3)

[11:43:12] Stdout logging level is INFO3.
[11:43:12] Task: binary

[11:43:12] Start automl preset with listed constraints:
[11:43:12] - time: 300.00 seconds
[11:43:12] - CPU: 4 cores
[11:43:12] - memory: 16 GB

[11:43:12] Train data shape: (8000, 122)

[11:43:13] Feats was rejected during automatic roles guess: []
[11:43:13] Layer 1 train process start. Time left 299.17 secs
[11:43:13] Training until validation scores don't improve for 200 rounds
[11:43:15] Selector_LightGBM fitting and predicting completed
[11:43:15] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[11:43:15] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[11:43:15] Training until validation scores don't improve for 200 rounds
[11:43:17] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[11:43:17] Training until validation scores don't improve for 200 rounds
[11:43:24] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[11:43:24] Training until validation scor

array([[0.08149866],
       [0.04592865],
       [0.04751563],
       ...,
       [0.06561954],
       [0.15983571],
       [0.11311316]], dtype=float32)